## 📦 Step 1: Load Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from datetime import datetime

print("✅ Libraries loaded successfully!")
base_dir = Path().resolve().parent.parent
os.chdir(base_dir)

## 🔓 Step 2: Load the Trained Model and Components

In [ ]:
print("📂 Loading model and preprocessing components...\n")

# Load the trained model
model = joblib.load('models/best_model.joblib')
print("✅ Model loaded")

# Load the scaler (used to standardize features)
scaler = joblib.load('models/scaler.joblib')
print("✅ Scaler loaded")

# Load feature names (to ensure correct order)
feature_names = joblib.load('models/feature_names.joblib')
print("✅ Feature names loaded")

# Load metadata (model information)
metadata = joblib.load('models/model_metadata.joblib')
print("✅ Metadata loaded")

print("\n" + "=" * 70)
print("📊 MODEL INFORMATION")
print("=" * 70)
print(f"Model Name: {metadata['model_name']}")
print(f"Model Type: {metadata['model_type']}")
print(f"Number of Features: {metadata['n_features']}")
print(f"Training Date: {metadata['training_date']}")
print(f"\nPerformance Metrics:")
for metric, value in metadata['performance'].items():
    if metric != 'Training Time':
        print(f"  • {metric}: {value:.4f}")
    else:
        print(f"  • {metric}: {value:.4f}s")

## 🎲 Step 3: Generate Synthetic Patient Data

Let's create realistic test data for demonstration purposes.

In [ ]:
print("🎲 Generating synthetic patient data...\n")

# Set random seed for reproducibility
np.random.seed(42)

# Number of synthetic patients to generate
n_patients = 20

# Generate realistic patient data
synthetic_data = {
    'Patient_ID': [f'P{i+1:03d}' for i in range(n_patients)],
    'Age [years]': np.random.uniform(25, 80, n_patients).round(1),
    'Height': np.random.uniform(150, 190, n_patients).round(1),
    'Weight': np.random.uniform(50, 100, n_patients).round(1),
    'BMI': None,  # Will calculate
    'Protein Concentration [mg/ml]': np.random.uniform(2.5, 7.5, n_patients).round(2),
    'Hydrophobicity [score]': np.random.uniform(10, 100, n_patients).round(1),
    'pH': np.random.uniform(6.5, 7.8, n_patients).round(2),
    'Glucose [mmol/L]': np.random.uniform(3.0, 8.5, n_patients).round(2),
    'Albumin [g/L]': np.random.uniform(30, 50, n_patients).round(1),
    'Sex': np.random.choice(['Male', 'Female'], n_patients),
    'Sample_Quality': np.random.choice(['Good', 'OK', 'Poor'], n_patients, p=[0.6, 0.3, 0.1])
}

# Create DataFrame
df_new = pd.DataFrame(synthetic_data)

# Calculate BMI
df_new['BMI'] = (df_new['Weight'] / (df_new['Height'] / 100) ** 2).round(1)

# Create engineered features (same as training)
df_new['Age_Group'] = pd.cut(df_new['Age [years]'], 
                              bins=[0, 30, 45, 60, 100], 
                              labels=['Young', 'Middle', 'Senior', 'Elderly'])

df_new['BMI_Category'] = pd.cut(df_new['BMI'], 
                                 bins=[0, 18.5, 25, 30, 100], 
                                 labels=['Underweight', 'Normal', 'Overweight', 'Obese'])

df_new['Protein_Albumin_Ratio'] = df_new['Protein Concentration [mg/ml]'] / df_new['Albumin [g/L]']

df_new['pH_Status'] = pd.cut(df_new['pH'], 
                              bins=[0, 6.8, 7.2, 14], 
                              labels=['Acidic', 'Normal', 'Alkaline'])

df_new['Glucose_Level'] = pd.cut(df_new['Glucose [mmol/L]'], 
                                  bins=[0, 5.5, 7, 20], 
                                  labels=['Normal', 'Prediabetic', 'Diabetic'])

df_new['Is_Good_Quality'] = df_new['Sample_Quality'].apply(lambda x: 1 if x == 'Good' else 0)

print(f"✅ Generated {n_patients} synthetic patients")
print(f"\n📊 Sample Data (first 5 patients):")
display(df_new[['Patient_ID', 'Age [years]', 'Sex', 'BMI', 'Glucose [mmol/L]', 'pH']].head())

## 🔄 Step 4: Prepare Data for Prediction

We need to encode categorical variables and ensure features match the training data.

In [ ]:
print("🔄 Preparing data for prediction...\n")

# Encode categorical variables
categorical_features = ['Sex', 'Age_Group', 'BMI_Category', 'pH_Status', 'Glucose_Level']
df_encoded = df_new.copy()

for col in categorical_features:
    if col in df_encoded.columns:
        dummies = pd.get_dummies(df_encoded[col], prefix=col, drop_first=True)
        df_encoded = pd.concat([df_encoded, dummies], axis=1)

# Ensure we have all the features the model expects
# Create a DataFrame with all required features
X_new = pd.DataFrame(columns=feature_names)

# Fill in the values we have
for col in feature_names:
    if col in df_encoded.columns:
        X_new[col] = df_encoded[col].values
    else:
        # If a feature is missing (e.g., a categorical level), fill with 0
        X_new[col] = 0

# Convert to numeric
X_new = X_new.apply(pd.to_numeric, errors='coerce').fillna(0)

print(f"✅ Data prepared with {X_new.shape[1]} features")
print(f"✅ Data shape: {X_new.shape}")
print(f"\n📋 Feature names match: {list(X_new.columns) == feature_names}")

## 🎯 Step 5: Make Predictions

In [ ]:
print("🎯 Making predictions...\n")

# Scale the features (using the same scaler from training)
X_new_scaled = scaler.transform(X_new)
print("✅ Features scaled")

# Make predictions
predictions = model.predict(X_new_scaled)
print("✅ Predictions generated")

# Get prediction probabilities (if available)
if hasattr(model, 'predict_proba'):
    probabilities = model.predict_proba(X_new_scaled)
    print("✅ Prediction probabilities calculated")
else:
    probabilities = None
    print("⚠️  Model doesn't support probability predictions")

# Add predictions to the dataframe
df_new['Prediction'] = predictions
df_new['Diagnosis_Predicted'] = df_new['Prediction'].apply(lambda x: 'Tumor' if x == 1 else 'Healthy')

if probabilities is not None:
    df_new['Probability_Healthy'] = probabilities[:, 0]
    df_new['Probability_Tumor'] = probabilities[:, 1]
    df_new['Confidence'] = df_new[['Probability_Healthy', 'Probability_Tumor']].max(axis=1)

print("\n" + "=" * 70)
print("📊 PREDICTION SUMMARY")
print("=" * 70)
print(f"Total patients: {len(df_new)}")
print(f"Predicted Healthy: {(df_new['Prediction'] == 0).sum()}")
print(f"Predicted Tumor: {(df_new['Prediction'] == 1).sum()}")

if probabilities is not None:
    print(f"\nAverage confidence: {df_new['Confidence'].mean():.2%}")
    print(f"Min confidence: {df_new['Confidence'].min():.2%}")
    print(f"Max confidence: {df_new['Confidence'].max():.2%}")

## 📊 Step 6: Display Prediction Results

In [ ]:
print("📋 DETAILED PREDICTION RESULTS\n")
print("=" * 90)

# Select columns to display
if probabilities is not None:
    display_cols = ['Patient_ID', 'Age [years]', 'Sex', 'BMI', 'Glucose [mmol/L]', 
                    'Diagnosis_Predicted', 'Probability_Healthy', 'Probability_Tumor', 'Confidence']
else:
    display_cols = ['Patient_ID', 'Age [years]', 'Sex', 'BMI', 'Glucose [mmol/L]', 'Diagnosis_Predicted']

results_display = df_new[display_cols].copy()

# Format probabilities as percentages
if probabilities is not None:
    results_display['Probability_Healthy'] = results_display['Probability_Healthy'].apply(lambda x: f"{x:.1%}")
    results_display['Probability_Tumor'] = results_display['Probability_Tumor'].apply(lambda x: f"{x:.1%}")
    results_display['Confidence'] = results_display['Confidence'].apply(lambda x: f"{x:.1%}")

display(results_display)

## 📈 Step 7: Visualize Predictions

In [ ]:
print("📈 Creating visualizations...\n")

fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Prediction Distribution
diagnosis_counts = df_new['Diagnosis_Predicted'].value_counts()
colors = ['green', 'red']
axes[0, 0].bar(diagnosis_counts.index, diagnosis_counts.values, color=colors)
axes[0, 0].set_title('Prediction Distribution', fontweight='bold', fontsize=14)
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_xlabel('Diagnosis')
for i, v in enumerate(diagnosis_counts.values):
    axes[0, 0].text(i, v + 0.3, str(v), ha='center', fontweight='bold', fontsize=12)

# Plot 2: Confidence Distribution
if probabilities is not None:
    axes[0, 1].hist(df_new['Confidence'], bins=15, color='steelblue', edgecolor='black', alpha=0.7)
    axes[0, 1].set_title('Prediction Confidence Distribution', fontweight='bold', fontsize=14)
    axes[0, 1].set_xlabel('Confidence Score')
    axes[0, 1].set_ylabel('Frequency')
    axes[0, 1].axvline(df_new['Confidence'].mean(), color='red', linestyle='--', 
                       linewidth=2, label=f'Mean: {df_new["Confidence"].mean():.2%}')
    axes[0, 1].legend()
else:
    axes[0, 1].text(0.5, 0.5, 'Confidence scores\nnot available', 
                    ha='center', va='center', fontsize=14, transform=axes[0, 1].transAxes)
    axes[0, 1].set_title('Prediction Confidence Distribution', fontweight='bold', fontsize=14)

# Plot 3: Age vs Diagnosis
for diagnosis in df_new['Diagnosis_Predicted'].unique():
    subset = df_new[df_new['Diagnosis_Predicted'] == diagnosis]
    color = 'green' if diagnosis == 'Healthy' else 'red'
    axes[1, 0].scatter(subset['Age [years]'], subset['BMI'], 
                       label=diagnosis, alpha=0.6, s=100, color=color, edgecolors='black')
axes[1, 0].set_title('Age vs BMI by Predicted Diagnosis', fontweight='bold', fontsize=14)
axes[1, 0].set_xlabel('Age (years)')
axes[1, 0].set_ylabel('BMI')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# Plot 4: Glucose vs Protein by Diagnosis
for diagnosis in df_new['Diagnosis_Predicted'].unique():
    subset = df_new[df_new['Diagnosis_Predicted'] == diagnosis]
    color = 'green' if diagnosis == 'Healthy' else 'red'
    axes[1, 1].scatter(subset['Glucose [mmol/L]'], subset['Protein Concentration [mg/ml]'], 
                       label=diagnosis, alpha=0.6, s=100, color=color, edgecolors='black')
axes[1, 1].set_title('Glucose vs Protein by Predicted Diagnosis', fontweight='bold', fontsize=14)
axes[1, 1].set_xlabel('Glucose (mmol/L)')
axes[1, 1].set_ylabel('Protein Concentration (mg/ml)')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Visualizations created!")

## 🔍 Step 8: Highlight High-Risk Patients

Identify patients with tumor predictions and high confidence.

In [ ]:
print("⚠️  HIGH-RISK PATIENTS (Tumor Predictions)\n")
print("=" * 90)

high_risk = df_new[df_new['Diagnosis_Predicted'] == 'Tumor'].copy()

if len(high_risk) > 0:
    if probabilities is not None:
        high_risk = high_risk.sort_values('Probability_Tumor', ascending=False)
        display_cols = ['Patient_ID', 'Age [years]', 'Sex', 'BMI', 'Glucose [mmol/L]', 
                        'Protein Concentration [mg/ml]', 'pH', 'Probability_Tumor']
    else:
        display_cols = ['Patient_ID', 'Age [years]', 'Sex', 'BMI', 'Glucose [mmol/L]', 
                        'Protein Concentration [mg/ml]', 'pH']
    
    print(f"Found {len(high_risk)} patients with tumor predictions:\n")
    display(high_risk[display_cols])
    
    print("\n💡 These patients should be prioritized for further medical evaluation.")
else:
    print("✅ No high-risk patients identified in this batch.")
    print("   All patients predicted as Healthy.")

## 💾 Step 9: Save Predictions to File

In [ ]:
print("💾 Saving predictions to file...\n")

# Create output filename with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f'predictions_{timestamp}.csv'

# Select columns to save
if probabilities is not None:
    save_cols = ['Patient_ID', 'Age [years]', 'Sex', 'Height', 'Weight', 'BMI',
                 'Protein Concentration [mg/ml]', 'Glucose [mmol/L]', 'pH', 'Albumin [g/L]',
                 'Diagnosis_Predicted', 'Probability_Healthy', 'Probability_Tumor', 'Confidence']
else:
    save_cols = ['Patient_ID', 'Age [years]', 'Sex', 'Height', 'Weight', 'BMI',
                 'Protein Concentration [mg/ml]', 'Glucose [mmol/L]', 'pH', 'Albumin [g/L]',
                 'Diagnosis_Predicted']

df_new[save_cols].to_csv(output_filename, index=False)

print(f"✅ Predictions saved to '{output_filename}'")
print(f"📊 Saved {len(df_new)} patient predictions")
print(f"\n💡 You can open this file in Excel or any CSV viewer.")

## 🎓 Summary

### What We Accomplished:

✅ **Loaded** the trained model and preprocessing components  
✅ **Generated** realistic synthetic patient data  
✅ **Prepared** data with proper feature engineering  
✅ **Made** predictions with confidence scores  
✅ **Visualized** prediction results  
✅ **Identified** high-risk patients  
✅ **Saved** predictions to a CSV file  

---

### 🚀 Next Steps:

1. **Use Real Data**: Replace synthetic data with actual patient records
2. **Batch Processing**: Process multiple files automatically
3. **API Integration**: Create a web service for real-time predictions
4. **Model Monitoring**: Track prediction distributions over time
5. **Validation**: Compare predictions with actual diagnoses when available

---

### 💡 Key Points:

- Always use the **same preprocessing** (scaling, encoding) as training
- Keep feature names and order **consistent**
- Save both **model and preprocessing components**
- Document model **metadata** for reproducibility
- Prioritize high-risk predictions for **medical review**

---